# import

In [23]:
import os
import sys

sys.path.append("../input/timmmaster/")

import argparse
import gc
import glob

import albumentations as A
import cv2
import dicomsdl
import joblib
import numpy as np
import pandas as pd
import pydicom
import pytorch_lightning as pl
import re
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import wandb
import yaml
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from tqdm import tqdm

from pydicom.filebase import DicomBytesIO
import pydicom

In [24]:
with open("/kaggle/input/rsna-exp296/exp296.yaml", encoding="utf-8") as f:
    cfg1 = yaml.safe_load(f)
cfg1["general"]["wandb_desabled"] = True
cfg1["general"]["input_path"] = "/kaggle/input/rsna-breast-cancer-detection"
cfg1["general"]["output_path"] = "/kaggle/input"
cfg1["general"]["save_name"] = "rsna-exp296"
cfg1["model"]["model_save"] = False
cfg1["pl_params"]["enable_checkpointing"] = False
cfg1["pl_params"]["precision"] = 16
cfg1["general"]["fold"] = None
cfg1["tta"] = False
cfg1["general"]["cv"] = False

with open("/kaggle/input/rsna-exp302/exp302.yaml", encoding="utf-8") as f:
    cfg2 = yaml.safe_load(f)
cfg2["general"]["wandb_desabled"] = True
cfg2["general"]["input_path"] = "/kaggle/input/rsna-breast-cancer-detection"
cfg2["general"]["output_path"] = "/kaggle/input"
cfg2["general"]["save_name"] = "rsna-exp302"
cfg2["model"]["model_save"] = False
cfg2["pl_params"]["enable_checkpointing"] = False
cfg2["pl_params"]["precision"] = 16
cfg2["general"]["fold"] = None
cfg2["tta"] = False
cfg2["general"]["cv"] = False

with open("/kaggle/input/rsna-exp288/exp288.yaml", encoding="utf-8") as f:
    cfg3 = yaml.safe_load(f)
cfg3["general"]["wandb_desabled"] = True
cfg3["general"]["input_path"] = "/kaggle/input/rsna-breast-cancer-detection"
cfg3["general"]["output_path"] = "/kaggle/input"
cfg3["general"]["save_name"] = "rsna-exp288"
cfg3["model"]["model_save"] = False
cfg3["pl_params"]["enable_checkpointing"] = False
cfg3["pl_params"]["precision"] = 16
cfg3["general"]["fold"] = None
cfg3["tta"] = False
cfg3["general"]["cv"] = False

cfg_list = [cfg1, cfg2, cfg3]

In [60]:
GPU = True
USE_DATA = "test"
BINARIZED = True
AGG = "top3_mean" # mean or max
THR = 0.44

In [26]:
if GPU:
    import nvjpeg2k
    decoder = nvjpeg2k.Decoder()
else:
    decoder = None

# Data Preparation

In [27]:
if USE_DATA == "train":
    csv_file = "/kaggle/input/rsna-breast-cancer-detection/train.csv"
    dcm_dir  = "/kaggle/input/rsna-breast-cancer-detection/train_images"

if USE_DATA == "test":
    csv_file = "/kaggle/input/rsna-breast-cancer-detection/test.csv"
    dcm_dir  = "/kaggle/input/rsna-breast-cancer-detection/test_images"

SIZE = 1536
VOI = cfg1["task"]["voi"] # True or False
BIT = cfg1["task"]["bit"] # 8, 16
EXTENSION = cfg1["task"]["extend"]
N_JOBS = 2 if GPU else 4 # gpu 2, cpu 4

if VOI:
    SAVE_FOLDER = f"/kaggle/tmp/output/{EXTENSION}_{SIZE}_{BIT}bit_voi/"
else:
    SAVE_FOLDER = f"/kaggle/tmp/output/{EXTENSION}_{SIZE}_{BIT}bit/"
    
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [28]:
def make_transfer_syntax_uid(df, dcm_dir):
    machine_id_to_transfer = {}
    machine_id = df.machine_id.unique()
    for i in machine_id:
        d = df[df.machine_id == i].iloc[0]
        f = f"{dcm_dir}/{d.patient_id}/{d.image_id}.dcm"
        dicom = pydicom.dcmread(f)
        machine_id_to_transfer[i] = dicom.file_meta.TransferSyntaxUID
    return machine_id_to_transfer

In [29]:
def process(f, size=512, save_folder="", extension="png", force_slow=False):
    patient = f.split('/')[-2]
    image = f.split('/')[-1][:-4]
    
    meta = pydicom.dcmread(f, stop_before_pixels=True)
    if meta.file_meta.TransferSyntaxUID == "1.2.840.10008.1.2.4.90":
        if not force_slow:
            #print("jpeg2k")
            with open(f, "rb") as _f:
                raw = DicomBytesIO(_f.read())
                ds = pydicom.dcmread(raw)
            offset = ds.PixelData.find(b"\x00\x00\x00\x0C")
            hackedbitstream = bytearray()
            hackedbitstream.extend(ds.PixelData[offset:])
            img = decoder.decode(hackedbitstream)
        else:
            #print("pydicom")
            img = pydicom.dcmread(f).pixel_array
    else:
        #print("dicomsdl")
        dicom = dicomsdl.open(f)
        img = dicom.pixelData()
    
    if VOI:
        img = apply_voi_lut(img, meta)

    img = (img - img.min()) / (img.max() - img.min())

    if meta.PhotometricInterpretation == "MONOCHROME1":
        img = 1 - img

    #img = cv2.resize(img, (size, size))
    
    if BIT == 16:
        img = (img * 65535).astype(np.uint16)
    else:
        img = (img * 255).astype(np.uint8)

    fit_image(img, patient, image, extension)

In [30]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_LINEAR):

    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)

    return resized

In [31]:
def fit_image(X, patient_id, im_id, extension, return_image=False):
    #X = cv2.imread(fname)
    
    # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
    X = X[5:-5, 5:-5]
    
    # regions of non-empty pixels
    output= cv2.connectedComponentsWithStats((X > 20).astype(np.uint8), 8, cv2.CV_32S)

    # stats.shape == (N, 5), where N is the number of regions, 5 dimensions correspond to:
    # left, top, width, height, area_size
    stats = output[2]

    # finding max area which always corresponds to the breast data. 
    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h
    
    # cutting out the breast data
    X_fit = X[y1: y2, x1: x2]
    
    #patient_id, im_id = re.findall("(\d+)_(\d+).png", os.path.basename(fname))[0]
    image = X_fit
    
    shape = image.shape
    if shape[0] > shape[1]:
        image = image_resize(image, height=SIZE)
    else:
        image = image_resize(image, width=SIZE)
        
    if return_image:
        return image
    else:
        cv2.imwrite(SAVE_FOLDER + f"{patient_id}_{im_id}.{extension}", image)

#def fit_all_images(all_images):
#    with ProcessPoolExecutor(4) as p:
#        for i in tqdm(p.map(fit_image, all_images), total=len(all_images)):
#            pass

In [32]:
test_df = pd.read_csv(csv_file)
machine_id_to_transfer = make_transfer_syntax_uid(test_df, dcm_dir)
test_df.loc[:, "i"] = np.arange(len(test_df))
test_df.loc[:, "TransferSyntaxUID"] = test_df.machine_id.map(machine_id_to_transfer)    

print("test_df", test_df.shape)
print(test_df)
print("")

test_df (4, 11)
   site_id  patient_id    image_id laterality view  age  implant  machine_id  \
0        2       10008   736471439          L  MLO   81        0          21   
1        2       10008  1591370361          L   CC   81        0          21   
2        2       10008    68070693          R  MLO   81        0          21   
3        2       10008   361203119          R   CC   81        0          21   

  prediction_id  i       TransferSyntaxUID  
0       10008_L  0  1.2.840.10008.1.2.4.90  
1       10008_L  1  1.2.840.10008.1.2.4.90  
2       10008_R  2  1.2.840.10008.1.2.4.90  
3       10008_R  3  1.2.840.10008.1.2.4.90  



In [33]:
j2k_df = test_df[test_df.TransferSyntaxUID == '1.2.840.10008.1.2.4.90'].reset_index(drop=True)
non_j2k_df = test_df[test_df.TransferSyntaxUID != '1.2.840.10008.1.2.4.90'].reset_index(drop=True)

In [34]:
if GPU:
    print("jpeg2k")
    for t, d in tqdm(j2k_df.iterrows()):
        dcm_file = f"{dcm_dir}/{d.patient_id}/{d.image_id}.dcm"
        process(dcm_file, size=SIZE, save_folder=SAVE_FOLDER, extension=EXTENSION, force_slow=False)
else:
    print("pydicom")
    Parallel(n_jobs=N_JOBS, backend="multiprocessing")(
        delayed(process)(f"{dcm_dir}/{d.patient_id}/{d.image_id}.dcm", size=SIZE, save_folder=SAVE_FOLDER, extension=EXTENSION, force_slow=True)
        for t, d in tqdm(j2k_df.iterrows())
    )

jpeg2k


4it [00:01,  3.41it/s]


In [35]:
Parallel(n_jobs=N_JOBS, backend="multiprocessing")(
    delayed(process)(f"{dcm_dir}/{d.patient_id}/{d.image_id}.dcm", size=SIZE, save_folder=SAVE_FOLDER, extension=EXTENSION)
    for t, d in tqdm(non_j2k_df.iterrows())
)

0it [00:00, ?it/s]


[]

In [36]:
os.listdir(SAVE_FOLDER)

['30843_1410376934.png',
 '32936_977536954.png',
 '41873_2031943229.png',
 '41023_429046294.png',
 '1550_2033444912.png',
 '33920_703119224.png',
 '21687_1639171991.png',
 '26670_687467082.png',
 '33084_237482176.png',
 '28893_1126631940.png',
 '10984_1561152908.png',
 '40923_732298748.png',
 '20690_269603241.png',
 '17787_259787163.png',
 '2850_60823005.png',
 '2162_2038045719.png',
 '25565_321066032.png',
 '37467_243709522.png',
 '21788_37549957.png',
 '26740_1043344718.png',
 '12503_1521661976.png',
 '14596_923539679.png',
 '35842_389965548.png',
 '314_2008895976.png',
 '16124_1079271172.png',
 '30739_1160857934.png',
 '40885_1695096889.png',
 '32533_1154407860.png',
 '16312_596642787.png',
 '38436_171703432.png',
 '22403_1662359882.png',
 '34001_1442743870.png',
 '20954_1502690495.png',
 '12635_1574511944.png',
 '38558_216962453.png',
 '1928_2133896069.png',
 '41692_734284346.png',
 '41126_1914438265.png',
 '1759_1811195939.png',
 '25250_68845569.png',
 '42760_1654108567.png',
 '35

# def class and function

### model

In [37]:
def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if labels[idx]:
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + 1e-7)
    c_recall = ctp / (y_true_count + 1e-7)
    if c_precision > 0 and c_recall > 0:
        result = (
            (1 + beta_squared)
            * (c_precision * c_recall)
            / (beta_squared * c_precision + c_recall + 1e-7)
        )
        return result
    else:
        return 0

def optimal_f1(labels, predictions):
    thres = np.arange(0, 1, 0.01)
    f1s = [pfbeta(labels, predictions > thr) for thr in thres]
    idx = np.argmax(f1s)
    return f1s[idx], thres[idx]
    
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True, pos_weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False, pos_weight=self.pos_weight)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
        cos(theta + m)
    """

    def __init__(
        self,
        in_features: int,
        out_features: int,
        s: float,
        m: float,
        easy_margin: bool,
        ls_eps: float,
    ):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input: torch.Tensor, label: torch.Tensor, device: str = "cuda") -> torch.Tensor:
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        # Enable 16 bit precision
        cosine = cosine.to(torch.float32)

        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [38]:
class CNNModel(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        if cfg["arcface"] is None or cfg["task"]["aux_target"] is None:
            num_classes = 1
        else:
            num_classes = None
        self.model = timm.create_model(
            model_name=cfg["model"]["model_name"],
            pretrained=False, #cfg["model"]["pretrained"],
            in_chans=cfg["model"]["in_chans"],
            num_classes=num_classes,
            drop_rate=cfg["model"]["drop_rate"],
            drop_path_rate=cfg["model"]["drop_path_rate"],
        )
        
        if cfg["model"]["criterion"] == "FocalLoss":
            self.criterion = FocalLoss(logits=True, pos_weight=cfg["model"]["loss_weights"])
        else:
            self.criterion = nn.__dict__[cfg["model"]["criterion"]](pos_weight=cfg["model"]["loss_weights"])
            
        if cfg["arcface"] is not None:
            self.embedding = nn.Linear(self.model.get_classifier().in_features, cfg["arcface"]["params"]["in_features"])
            self.arc = ArcMarginProduct(**cfg["arcface"]["params"])
            self.criterion = nn.CrossEntropyLoss()
            self.model.reset_classifier(num_classes=0, global_pool="avg")
        elif cfg["task"]["aux_target"] is not None:
            self.nn_cancer = torch.nn.Sequential(torch.nn.Linear(self.model.get_classifier().in_features, 1))
            self.nn_aux = torch.nn.ModuleList([torch.nn.Linear(self.model.get_classifier().in_features, n) for n in cfg["task"]["aux_target_nclasses"]])
            self.model.reset_classifier(num_classes=0, global_pool="avg")
        
        if cfg["model"]["grad_checkpointing"]:
            print("grad_checkpointing true")
            self.model.set_grad_checkpointing(enable=True)

    def forward(self, X):
        if self.cfg["arcface"] is not None:
            X = self.model(X)
            outputs = self.embedding(X)
        elif self.cfg["task"]["aux_target"] is not None:
            X = self.model(X)
            cancer = self.nn_cancer(X) #.squeeze()
            aux = []
            for nn in self.nn_aux:
                aux.append(nn(X)) #.squeeze()
            return cancer, aux
        else:
            outputs = self.model(X)
        return outputs
    
    def rand_bbox(self, size, lam):
        W = size[2]
        H = size[3]
        cut_rat = np.sqrt(1. - lam)
        cut_w = np.int(W * cut_rat)
        cut_h = np.int(H * cut_rat)

        # uniform
        cx = np.random.randint(W)
        cy = np.random.randint(H)

        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)
        return bbx1, bby1, bbx2, bby2

    def cutmix_data(self, x, y, alpha=1.0):
        indices = torch.randperm(x.size(0))
        shuffled_data = x[indices]
        shuffled_target = y[indices]

        lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4)
        bbx1, bby1, bbx2, bby2 = self.rand_bbox(x.size(), lam)
        new_data = x.clone()
        new_data[:, :, bby1:bby2, bbx1:bbx2] = x[indices, :, bby1:bby2, bbx1:bbx2]
        # adjust lambda to exactly match pixel ratio
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))

        return new_data, y, shuffled_target, lam
    
    def mixup_data(self, x, y, alpha=1.0, return_index=False):
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        batch_size = x.size()[0]
        index = torch.randperm(batch_size)
        mixed_x = lam * x + (1 - lam) * x[index, :]
        y_a, y_b = y, y[index]
        
        if return_index:
            return mixed_x, y_a, y_b, lam, index
        else:
            return mixed_x, y_a, y_b, lam
    
    def mix_criterion(self, pred, y_a, y_b, lam, criterion="default"):
        if criterion == "default":
            criterion = self.criterion
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

    def training_step(self, batch, batch_idx):
        if self.cfg["model"]["train_2nd"] and self.current_epoch >= (self.cfg["pl_params"]["max_epochs"] - self.cfg["model"]["epoch_2nd"]):
            # 最後だけaugmentation切るとかする用
            self.cfg["model"]["aug_mix"] = False      
        if self.cfg["model"]["aug_mix"] and torch.rand(1) < 0.5:
            if self.cfg["arcface"] is not None:
                X, y = batch
                mixed_X, y_a, y_b, lam = self.mixup_data(X, y)
                pred_y = self.forward(mixed_X)
                pred_y = self.arc(pred_y, y, self.device)
                loss = self.mix_criterion(pred_y ,y_a.long().squeeze(), y_b.long().squeeze(), lam)
                self.log("train_loss", loss, prog_bar=True)
                return loss
            elif self.cfg["task"]["aux_target"] is not None:
                X, y, aux_y = batch
                mixed_X, y_a, y_b, lam, index = self.mixup_data(X, y, return_index=True)
                pred_y, pred_aux_y = self.forward(mixed_X)
                aux_y_a, aux_y_b = aux_y, aux_y[index]
                cancer_loss = self.mix_criterion(pred_y ,y_a, y_b, lam)
                aux_loss = torch.mean(torch.stack([self.mix_criterion(pred_aux_y[i], aux_y_a[:, i], aux_y_b[:, i], lam, criterion=torch.nn.functional.cross_entropy) for i in range(aux_y.shape[-1])]))
                loss = cancer_loss + self.cfg["task"]["aux_loss_weight"] * aux_loss
                self.log("cancer_loss", cancer_loss, prog_bar=False)
                self.log("aux_loss", aux_loss, prog_bar=False)
                self.log("train_loss", loss, prog_bar=True)
                return {"loss": loss, "cancer_loss": cancer_loss, "aux_loss": aux_loss}
            else:
                X, y = batch
                #if torch.rand(1) >= 0.5:
                #    mixed_X, y_a, y_b, lam = self.mixup_data(X, y)
                #else:
                #    mixed_X, y_a, y_b, lam = self.cutmix_data(X, y)
                mixed_X, y_a, y_b, lam = self.mixup_data(X, y)
                pred_y = self.forward(mixed_X)
                loss = self.mix_criterion(pred_y ,y_a, y_b, lam)
                self.log("train_loss", loss, prog_bar=True)
                return loss
        else:
            if self.cfg["arcface"] is not None:
                X, y = batch
                pred_y = self.arc(pred_y, y, self.device)
                loss = self.criterion(pred_y, y.long().squeeze())
                self.log("train_loss", loss, prog_bar=True)
                return loss
            elif self.cfg["task"]["aux_target"] is not None:
                X, y, aux_y = batch
                pred_y, pred_aux_y = self.forward(X)
                cancer_loss = self.criterion(pred_y, y)
                aux_loss = torch.mean(torch.stack([torch.nn.functional.cross_entropy(pred_aux_y[i], aux_y[:, i]) for i in range(aux_y.shape[-1])]))
                loss = cancer_loss + self.cfg["task"]["aux_loss_weight"] * aux_loss
                self.log("cancer_loss", cancer_loss, prog_bar=False)
                self.log("aux_loss", aux_loss, prog_bar=False)
                self.log("train_loss", loss, prog_bar=True)
                return {"loss": loss, "cancer_loss": cancer_loss, "aux_loss": aux_loss}
            else:
                X, y = batch
                pred_y = self.forward(X)
                loss = self.criterion(pred_y, y)
                self.log("train_loss", loss, prog_bar=True)
                return loss

    def training_epoch_end(self, outputs):
        loss_list = [x["loss"] for x in outputs]
        avg_loss = torch.stack(loss_list).mean()
        self.log("train_avg_loss", avg_loss, prog_bar=True)
        
        if self.cfg["task"]["aux_target"] is not None:
            cancer_loss_list = [x["cancer_loss"] for x in outputs]
            aux_loss_list = [x["aux_loss"] for x in outputs]
            cancer_avg_loss = torch.stack(cancer_loss_list).mean()
            aux_avg_loss = torch.stack(aux_loss_list).mean()
            self.log("train_avg_cancer_loss", cancer_avg_loss, prog_bar=False)
            self.log("train_avg_aux_loss", aux_avg_loss, prog_bar=False)

    def validation_step(self, batch, batch_idx):
        if self.cfg["arcface"] is not None:
            X, y = batch
            pred_y = self.forward(X)
            pred_y = self.arc(pred_y, y, self.device)
            loss = self.criterion(pred_y, y.long().squeeze())
            pred_y = nn.Softmax(dim=1)(pred_y)
            pred_y = pred_y[:, 1]
            pred_y = torch.nan_to_num(pred_y)
            return {"valid_loss": loss, "preds": pred_y, "targets": y}
        elif self.cfg["task"]["aux_target"] is not None:
            X, y, aux_y = batch
            pred_y, pred_aux_y = self.forward(X)
            cancer_loss = self.criterion(pred_y, y)
            aux_loss = torch.mean(torch.stack([torch.nn.functional.cross_entropy(pred_aux_y[i], aux_y[:, i]) for i in range(aux_y.shape[-1])]))
            loss = cancer_loss + self.cfg["task"]["aux_loss_weight"] * aux_loss
            pred_y = torch.sigmoid(pred_y)
            pred_y = torch.nan_to_num(pred_y)
            return {"valid_loss": loss, "cancer_loss": cancer_loss, "aux_loss": aux_loss, "preds": pred_y, "targets": y}
        else:
            X, y = batch
            pred_y = self.forward(X)
            loss = self.criterion(pred_y, y)
            pred_y = torch.sigmoid(pred_y)
            pred_y = torch.nan_to_num(pred_y)
            return {"valid_loss": loss, "preds": pred_y, "targets": y}

    def validation_epoch_end(self, outputs):
        loss_list = [x["valid_loss"] for x in outputs]
        preds = torch.cat([x["preds"] for x in outputs], dim=0).cpu().detach().numpy()
        targets = (
            torch.cat([x["targets"] for x in outputs], dim=0).cpu().detach().numpy()
        )
        avg_loss = torch.stack(loss_list).mean()
        pfbeta_score = pfbeta(targets.flatten(), preds.flatten())
        if np.unique(targets).shape[0] == 1:
            auc_score = 0.0
        else:
            auc_score = sklearn.metrics.roc_auc_score(targets.flatten(), preds.flatten())
        optimized_pfbeta_score, threshold = optimal_f1(targets.flatten(), preds.flatten())
        recall = sklearn.metrics.recall_score(targets.flatten(), preds.flatten() > threshold)
        specificity = sklearn.metrics.recall_score(targets.flatten(), preds.flatten() > threshold, pos_label=0)
        precision = sklearn.metrics.precision_score(targets.flatten(), preds.flatten() > threshold)
        self.log("valid_avg_loss", avg_loss, prog_bar=True)
        self.log("valid_pfbeta_score", pfbeta_score, prog_bar=True)
        self.log("optimized_pfbeta_score", optimized_pfbeta_score, prog_bar=True)
        self.log("valid_auc_score", auc_score, prog_bar=True)
        self.log("threshold", threshold, prog_bar=False)
        self.log("recall", recall, prog_bar=False)
        self.log("specificity", specificity, prog_bar=False)
        self.log("precision", precision, prog_bar=False)
        
        if self.cfg["task"]["aux_target"] is not None:
            cancer_loss_list = [x["cancer_loss"] for x in outputs]
            aux_loss_list = [x["aux_loss"] for x in outputs]
            cancer_avg_loss = torch.stack(cancer_loss_list).mean()
            aux_avg_loss = torch.stack(aux_loss_list).mean()
            self.log("valid_avg_cancer_loss", cancer_avg_loss, prog_bar=False)
            self.log("valid_avg_aux_loss", aux_avg_loss, prog_bar=False)
        
        return avg_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if self.cfg["tta"]:
            X, X_2, y = batch
            
            
            if self.cfg["arcface"] is not None:
                pred_y_1 = self.forward(X)
                pred_y_2 = self.forward(X_2)
                pred_y_1 = self.arc(pred_y_1, y, self.device)      
                pred_y_2 = self.arc(pred_y_2, y, self.device) 
                pred_y_1 = nn.Softmax(dim=1)(pred_y_1)
                pred_y_2 = nn.Softmax(dim=1)(pred_y_2)
                pred_y_1 = pred_y_1[:, 1]
                pred_y_2 = pred_y_2[:, 1]
            elif self.cfg["task"]["aux_target"] is not None:
                pred_y_1, _ = self.forward(X)
                pred_y_2, _ = self.forward(X_2)
                pred_y_1 = torch.sigmoid(pred_y_1)
                pred_y_2 = torch.sigmoid(pred_y_2)
            else:
                pred_y_1 = self.forward(X)
                pred_y_2 = self.forward(X_2)
                pred_y_1 = torch.sigmoid(pred_y_1)
                pred_y_2 = torch.sigmoid(pred_y_2)
                
            pred_y = (pred_y_1 + pred_y_2) / 2.0
        else:
            X, y = batch
            
            if self.cfg["arcface"] is not None:
                pred_y = self.forward(X)
                pred_y = self.arc(pred_y, y, self.device)        
                pred_y = nn.Softmax(dim=1)(pred_y)
                pred_y = pred_y[:, 1]
            elif self.cfg["task"]["aux_target"] is not None:
                pred_y, _ = self.forward(X)
                pred_y = torch.sigmoid(pred_y)
            else:
                pred_y = self.forward(X)
                pred_y = torch.sigmoid(pred_y)
                
        return pred_y

    def configure_optimizers(self):
        optimizer = optim.__dict__[self.cfg["model"]["optimizer"]["name"]](
            self.parameters(), **self.cfg["model"]["optimizer"]["params"]
        )
        if self.cfg["model"]["scheduler"] is None:
            return [optimizer]
        else:
            if self.cfg["model"]["scheduler"]["name"] == "OneCycleLR":
                scheduler = optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    #steps_per_epoch=self.cfg["len_train_loader"] // self.cfg["pl_params"]["accumulate_grad_batches"],
                    total_steps=self.trainer.estimated_stepping_batches,
                    **self.cfg["model"]["scheduler"]["params"],
                )
                scheduler = {"scheduler": scheduler, "interval": "step"}
            elif self.cfg["model"]["scheduler"]["name"] == "ReduceLROnPlateau":
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                    optimizer, **self.cfg["model"]["scheduler"]["params"],
                )
                scheduler = {
                    "scheduler": scheduler,
                    "interval": "epoch",
                    "monitor": "valid_avg_loss",
                }
            else:
                scheduler = optim.lr_scheduler.__dict__[
                    self.cfg["model"]["scheduler"]["name"]
                ](optimizer, **self.cfg["model"]["scheduler"]["params"])
            return [optimizer], [scheduler]

In [39]:
class RSNADataset(torch.utils.data.Dataset):
    def __init__(self, cfg, X, y=None, augmentation=False, cutout=False, aux=False):
        self.cfg = cfg
        self.augmentation = augmentation
        self.cutout = cutout
        self.df = X
        self.aux = aux
        
        if cfg["model"]["in_chans"] == 3:
            self.mean = cfg["model"]["mean"]
            self.std = cfg["model"]["std"]
        else:
            self.mean = cfg["model"]["mean"]
            self.std = cfg["model"]["std"]
        
        if y is None:
            self.y = torch.zeros(len(self.df), dtype=torch.float32)
        else:
            self.y = torch.tensor(y.values, dtype=torch.float32)
        
        # normalize
        self.normalize = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=self.mean, std=self.std),
            torchvision.transforms.Resize((cfg["task"]["height_size"], cfg["task"]["width_size"])),
            #torchvision.transforms.Normalize(mean=self.mean, std=self.std),
            
            #torchvision.transforms.Resize(min(cfg["task"]["height_size"], cfg["task"]["width_size"])),
            #torchvision.transforms.CenterCrop((cfg["task"]["height_size"], cfg["task"]["width_size"])),
        ])
        # resize
        #self.resize = torchvision.transforms.CenterCrop((cfg["task"]["height_size"], cfg["task"]["width_size"]))
        
        
        # augmentation
        # flip
        self.aug_hori_flip = A.HorizontalFlip(p=0.5)
        self.aug_ver_flip = A.VerticalFlip(p=0.5)
        # elastic and grid
        self.aug_distortion = A.GridDistortion(p=0.5)
        """
        A.OneOf([
            A.ElasticTransform(p=0.5),
            A.GridDistortion(p=0.5)
        ], p=0.5)
        """
        # affine
        #self.aug_affine = A.Affine(scale=(0.8, 1.2), translate_percent=(0.0, 0.1), rotate=(-45, 45), shear=(-15, 15), p=0.8)
        self.aug_affine = A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.8)
        # clahe
        self.aug_clahe = A.CLAHE(p=0.5)
        # bright
        self.aug_bright = A.OneOf([
            A.RandomGamma(gamma_limit=(50, 150), p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5)
        ], p=0.5)
        # cutout
        self.aug_cutout = A.CoarseDropout(max_height=8, max_width=8, p=0.5)
        # randomcrop
        #self.randomcrop = A.RandomResizedCrop(height=cfg["task"]["height_size"], width=cfg["task"]["width_size"], scale=(0.1, 1.0), ratio=(0.5, 1.0), p=0.8)

    def __len__(self):
        return len(self.df)
    
    def image_resize(self, image, width = None, height = None, inter = cv2.INTER_LINEAR):
        dim = None
        (h, w) = image.shape[:2]

        if width is None and height is None:
            return image

        if width is None:
            r = height / float(h)
            dim = (int(w * r), height)
        else:
            r = width / float(w)
            dim = (width, int(h * r))
        resized = cv2.resize(image, dim, interpolation = inter)

        return resized

    def __getitem__(self, index):
        image_id = self.df.loc[index, "image_id"]
        patient_id = self.df.loc[index, "patient_id"]
        extend = self.cfg["task"]["extend"]
        size = self.cfg["task"]["size"]
        bit = self.cfg["task"]["bit"]
        if self.cfg["task"]["voi"]:
            image_path = (
                f"{SAVE_FOLDER}/{patient_id}_{image_id}"
            )
        else:
            image_path = (
                f"{SAVE_FOLDER}/{patient_id}_{image_id}"
            )
            
        if self.cfg["model"]["in_chans"] == 3:
            X = cv2.imread(f"{image_path}.png")
        else:
            X = cv2.imread(f"{image_path}.png", cv2.IMREAD_GRAYSCALE)
        
        """
        shape = X.shape
        if shape[0] > shape[1]:
            X = self.image_resize(X, height=self.cfg["task"]["height_size"])
        else:
            X = self.image_resize(X, width=self.cfg["task"]["width_size"])
        X = Image.fromarray(X)
        X = self.resize(X)
        X = np.array(X)
        """
    
        # augmentation
        if self.augmentation:
            X = self.aug_hori_flip(image=X)["image"]
            X = self.aug_ver_flip(image=X)["image"]
            #X = self.aug_distortion(image=X)["image"]
            #X = self.aug_clahe(image=X)["image"]
            X = self.aug_affine(image=X)["image"]
            X = self.aug_bright(image=X)["image"]
            if self.cutout:
                X = self.aug_cutout(image=X)["image"]
            #X = self.randomcrop(image=X)["image"]
            
        y = self.y[index].unsqueeze(0)
            
        if self.cfg["tta"]:
            X_2 = self.aug_hori_flip(image=X)["image"]
            X = self.normalize(X)
            X_2 = self.normalize(X_2)
            return X, X_2, y
        else:
            X = self.normalize(X)
            #X = torchvision.transforms.ToTensor()(X)
            #if self.normalize:
            #    if self.cfg["model"]["normalize_method"] == "z_intensity":
            #        X = torchvision.transforms.Normalize(mean=torch.mean(X, dim=(1, 2)), std=torch.std(X, dim=(1, 2))+1e-7)(X)
            #    else:
            #        X = torchvision.transforms.Normalize(mean=self.mean, std=self.std)
            #X = torchvision.transforms.Resize((self.cfg["task"]["height_size"], self.cfg["task"]["width_size"]))(X)
            if self.aux:
                aux_y = self.df.iloc[index][self.cfg["task"]["aux_target"]]
                aux_y = torch.tensor(aux_y.values, dtype=torch.long)
                return X, y, aux_y
            else:
                return X, y

In [40]:
class CNNClassifierInference:
    def __init__(self, cfg, weight_path=None):
        # aux
        if cfg["task"]["aux_target"] is not None:
            cfg["task"]["aux_target_nclasses"] = []
            for t in cfg["task"]["aux_target"]:
                if t == "site_id":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "laterality":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "view":
                    cfg["task"]["aux_target_nclasses"].append(6)
                elif t == "implant":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "biopsy":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "invasive":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "BIRADS":
                    cfg["task"]["aux_target_nclasses"].append(4)
                elif t == "density":
                    cfg["task"]["aux_target_nclasses"].append(5)
                elif t == "difficult_negative_case":
                    cfg["task"]["aux_target_nclasses"].append(2)
                elif t == "age":
                    cfg["task"]["aux_target_nclasses"].append(10)
            print(cfg["task"]["aux_target_nclasses"])
        else:
            aux = False
            
        self.weight_path = weight_path
        self.cfg = cfg
        if cfg["model"]["weighted_loss"]:
            self.cfg["model"]["loss_weights"] = torch.tensor(0.0, dtype=torch.float32)
        else:
            self.cfg["model"]["loss_weights"] = None
        self.model = CNNModel(self.cfg)
        self.trainer = Trainer(**self.cfg["pl_params"])

    def predict(self, test_X):
        test_dataset = RSNADataset(self.cfg, test_X)
        test_dataloader = torch.utils.data.DataLoader(
            test_dataset, **self.cfg["test_loader"],
        )
        preds = self.trainer.predict(
            self.model, dataloaders=test_dataloader, ckpt_path=self.weight_path
        )
        preds = torch.cat(preds, axis=0)
        preds = preds.cpu().detach().numpy()

        return preds

In [41]:
def one_fold_test(cfg, test_X, fold_n):
    print(f"[fold_{fold_n}]")
    seed_everything(cfg["general"]["seed"], workers=True)

    model = CNNClassifierInference(cfg, f"{cfg['ckpt_path']}.ckpt")
    test_preds = model.predict(test_X)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return test_preds

In [42]:
def all_test(cfg, test_X):
    print(f"[all]")
    seed_everything(cfg["general"]["seed"], workers=True)

    model = CNNClassifierInference(cfg, f"{cfg['ckpt_path']}.ckpt")
    test_preds = model.predict(test_X)

    del model
    gc.collect()
    torch.cuda.empty_cache()

    return test_preds

# main

In [61]:
train = pd.read_csv(f"{cfg1['general']['input_path']}/train.csv")
train_X = train.drop(cfg1["task"]["target"], axis=1)

test_X = pd.read_csv(f"{cfg1['general']['input_path']}/test.csv")
sub = pd.read_csv(f"{cfg1['general']['input_path']}/sample_submission.csv")

In [62]:
if USE_DATA == "test":
    X = test_X
else:
    X = train_X

In [63]:
# random seed setting
seed_everything(cfg1["general"]["seed"], workers=True)

final_test_preds = []
if cfg1["general"]["cv"]:
    test_preds_list = []
    for fold_n in tqdm(cfg1["general"]["fold"]):
        for cfg in cfg_list:
            print(cfg["general"]["save_name"])
            cfg["fold_n"] = fold_n
            cfg["ckpt_path"] = f"{cfg['general']['output_path']}/{cfg['general']['save_name']}/last_epoch_fold{fold_n}"
            test_preds = one_fold_test(cfg, X, fold_n)
            test_preds_list.append(test_preds)
            print(test_preds)
            del test_preds
        final_test_preds.append(np.mean(test_preds_list, axis=0))
        print("-----------------")
        print("ensemble: ")
        print(np.mean(test_preds_list, axis=0))
        print("-----------------")
    final_test_preds = np.mean(final_test_preds, axis=0)
    print()
    print("final_test_preds:")
    print(final_test_preds)
else:
    test_preds_list = []
    for cfg in cfg_list:
        cfg["fold_n"] = "all"
        cfg["ckpt_path"] = f"{cfg['general']['output_path']}/{cfg['general']['save_name']}/last_epoch"
        test_preds = all_test(cfg, test_X)
        test_preds_list.append(test_preds)
        print(test_preds)
        del test_preds
    final_test_preds = np.mean(test_preds_list, axis=0)
    print()
    print("final_test_preds:")
    print(final_test_preds)

[all]
[2, 2, 6, 2, 2, 2, 4, 5, 2, 10]


Predicting: 0it [00:00, ?it/s]

[[0.002562]
 [0.1599  ]
 [0.006744]
 [0.558   ]]
[all]
[2, 2, 6, 2, 2, 2, 4, 5, 2, 10]


Predicting: 0it [00:00, ?it/s]

[[0.00923 ]
 [0.012924]
 [0.001421]
 [0.00701 ]]
[all]


Predicting: 0it [00:00, ?it/s]

[[0.00181]
 [0.06223]
 [0.00938]
 [0.04794]]

final_test_preds:
[[0.004536]
 [0.07837 ]
 [0.005848]
 [0.2043  ]]


In [64]:
sub = X.copy()
sub["cancer"] = final_test_preds

if AGG == "max":
    sub = sub[["prediction_id", "cancer"]].groupby("prediction_id").max().reset_index()
elif AGG == "mean":
    sub = sub[["prediction_id", "cancer"]].groupby("prediction_id").mean().reset_index()
elif AGG == "top3_mean":
    sub = sub[["prediction_id", "cancer"]].groupby("prediction_id").head(3).groupby("prediction_id").mean().reset_index()

In [47]:
test_X


,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [48]:
train_X

,site_id,patient_id,image_id,laterality,view,age,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0.0,0,C,216,True


In [49]:
sub

,prediction_id,cancer
0,10008_L,0.041443
1,10008_R,0.105103


In [50]:
if BINARIZED:
    sub["cancer"] = sub["cancer"] > THR
    sub = sub.astype({"cancer": float})

In [51]:
sub

,prediction_id,cancer
0,10008_L,0.0
1,10008_R,0.0


In [52]:
sub.to_csv('/kaggle/working/submission.csv', index=False)